This script is used to generate prediction given a timeseries or filtered the prediction and save the result on a file.
Computing the prediction or filtering it with DTW is time consuming. Instead of computing it each time, just one time.

In [1]:
# packages
import pandas as pd
import numpy as np
import math
import pickle
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from dtaidistance import dtw,similarity
from scipy.stats import pearsonr
from sklearn.model_selection import ParameterGrid

from claspy.segmentation import BinaryClaSPSegmentation

import ruptures as rpt

import stumpy
from aeon.segmentation import find_dominant_window_sizes

from aeon.segmentation import GreedyGaussianSegmenter

from aeon.segmentation import InformationGainSegmenter

from aeon.anomaly_detection import STRAY

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer,mean_squared_error
from ruptures.metrics import precision_recall
import matplotlib.pyplot as plt
#from aeon.visualisation import plot_series_with_change_points, plot_series_with_profiles
import seaborn as sns

sns.set_theme()
sns.set_color_codes()

from claspy.tests.evaluation import f_measure,covering

from claspy.window_size import dominant_fourier_frequency, highest_autocorrelation, suss

C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


Extracted features. Use the index of this list to use with iloc[]
<ol start="0">
  <li>Kinetic Global</li>
  <li>Kinetic Chest</li>
  <li>Directness Head</li>
  <li>Density</li>
  <li>left wrist ke</li>
  <li>right wrist ke</li>
  <li>left ankle ke</li>
  <li>right ankle ke</li>
  <li>head ke</li>
  <li>crouch density</li>
  <li>left leg density</li>
  <li>right leg density</li>
  <li>left hand density</li>
  <li>right hand density</li>
  <li>head density</li>
  <li>arto inferiore</li>
  <li>gamba</li>
  <li>coscia</li>
  <li>coscia dx</li>
  <li>coscia sx</li>
  <li>gamba sx</li>
  <li>gamba dx</li>
  <li>braccio sx</li>
  <li>braccio dx</li>
  <li>avambraccio sx</li>
  <li>avambraccio dx</li>
  <li>ARIEL speed magnitude</li>
  <li>ARIEL speed X component</li>
  <li>ARIEL speed Y component</li>
  <li>ARIEL speed Z component</li>
  <li>ARIEL acceleration magnitude</li>
  <li>ARIEL acceleration X component</li>
  <li>ARIEL acceleration Y component</li>
  <li>ARIEL acceleration Z component</li>
  <li>ARIEL jerk magnitude</li>
  <li>ARIEL jerk X component</li>
  <li>ARIEL jerk Y component</li>
  <li>ARIEL jerk Z component</li>
  <li>STRN speed magnitude</li>
  <li>STRN speed X component</li>
  <li>STRN speed Y component</li>
  <li>STRN speed Z component</li>
  <li>STRN acceleration magnitude</li>
  <li>STRN acceleration X component</li>
  <li>STRN acceleration Y component</li>
  <li>STRN accelerationZ component</li>
  <li>STRN jerk magnitude</li>
  <li>STRN jerk X component</li>
  <li>STRN jerk Y component</li>
  <li>STRN jerk Z component</li>
  <li>RHEL speed magnitude</li>
  <li>RHEL speed X component</li>
  <li>RHEL speed Y component</li>
  <li>RHEL speed Z component</li>
  <li>RHEL acceleration magnitude</li>
  <li>RHEL acceleration X component</li>
  <li>RHEL acceleration Y component</li>
  <li>RHEL acceleration Z component</li>
  <li>RHEL jerk magnitude</li>
  <li>RHEL jerk X component</li>
  <li>RHEL jerk Y component</li>
  <li>RHEL jerk Z component</li>
  <li>LHEL speed magnitude</li>
  <li>LHEL speed X component</li>
  <li>LHEL speed Y component</li>
  <li>LHEL speed Z component</li>
  <li>LHEL acceleration magnitude</li>
  <li>LHEL acceleration X component</li>
  <li>LHEL acceleration Y component</li>
  <li>LHEL acceleration Z component</li>
  <li>LHEL jerk magnitude</li>
  <li>LHEL jerk X component</li>
  <li>LHEL jerk Y component</li>
  <li>LHEL jerk Z component</li>
  <li>RPLM speed magnitude</li>
  <li>RPLM speed X component</li>
  <li>RPLM speed Y component</li>
  <li>RPLM speed Z component</li>
  <li>RPLM acceleration magnitude</li>
  <li>RPLM acceleration X component</li>
  <li>RPLM acceleration Y component</li>
  <li>RPLM acceleration Z component</li>
  <li>RPLM jerk magnitude</li>
  <li>RPLM jerk X component</li>
  <li>RPLM jerk Y component</li>
  <li>RPLM jerk Z component</li>
  <li>LPLM speed magnitude</li>
  <li>LPLM speed X component</li>
  <li>LPLM speed Y component</li>
  <li>LPLM speed Z component</li>
  <li>LPLM acceleration magnitude</li>
  <li>LPLM acceleration X component</li>
  <li>LPLM acceleration Y component</li>
  <li>LPLM acceleration Z component</li>
  <li>LPLM jerk magnitude</li>
  <li>LPLM jerk X component</li>
  <li>LPLM jerk Y component</li>
  <li>LPLM jerk Z component</li>




</ol>

In [2]:
# list of features. To access its name or its value while using iloc
features_name=[
    "kinetic_global",
    "kinetic_chest",
    "directness_head",
    "density",
    "left_wrist_ke",
    "right_wrist_ke",
    "left_ankle_ke",
    "right_ankle_ke",
    "head_ke",
    "crouch_density",
    "left_leg_density",
    "right_leg_density",
    "left_hand_density",
    "right_hand_density",
    "head_density",
    "arto_inferiore",
    "gamba",
    "coscia",
    "coscia_dx",
    "coscia_sx",
    "gamba_sx",
    "gamba_dx",
    "braccio_sx",
    "braccio_dx",
    "avambraccio_sx",
    "avambraccio_dx",
    "ARIEL_speed_magnitude",
    "ARIEL_speed_X_component",
    "ARIEL_speed_Y_component",
    "ARIEL_speed_Z_component",
    "ARIEL_acceleration_magnitude",
    "ARIEL_acceleration_X_component",
    "ARIEL_acceleration_Y_component",
    "ARIEL_acceleration_Z_component",
    "ARIEL_jerk_magnitude",
    "ARIEL_jerk_X_component",
    "ARIEL_jerk_Y_component",
    "ARIEL_jerk_Z_component",
    "STRN_speed_magnitude",
    "STRN_speed_X_component",
    "STRN_speed_Y_component",
    "STRN_speed_Z_component",
    "STRN_acceleration_magnitude",
    "STRN_acceleration_X_component",
    "STRN_acceleration_Y_component",
    "STRN_acceleration_Z_component",
    "STRN_jerk_magnitude",
    "STRN_jerk_X_component",
    "STRN_jerk_Y_component",
    "STRN_jerk_Z_component",
    "RHEL_speed_magnitude",
    "RHEL_speed_X_component",
    "RHEL_speed_Y_component",
    "RHEL_speed_Z_component",
    "RHEL_acceleration_magnitude",
    "RHEL_acceleration_X_component",
    "RHEL_acceleration_Y_component",
    "RHEL_acceleration_Z_component",
    "RHEL_jerk_magnitude",
    "RHEL_jerk_X_component",
    "RHEL_jerk_Y_component",
    "RHEL_jerk_Z_component",
    "LHEL_speed_magnitude",
    "LHEL_speed_X_component",
    "LHEL_speed_Y_component",
    "LHEL_speed_Z_component",
    "LHEL_acceleration_magnitude",
    "LHEL_acceleration_X_component",
    "LHEL_acceleration_Y_component",
    "LHEL_acceleration_Z_component",
    "LHEL_jerk_magnitude",
    "LHEL_jerk_X_component",
    "LHEL_jerk_Y_component",
    "LHEL_jerk_Z_component",
    "RPLM_speed_magnitude",
    "RPLM_speed_X_component",
    "RPLM_speed_Y_component",
    "RPLM_speed_Z_component",
    "RPLM_acceleration_magnitude",
    "RPLM_acceleration_X_component",
    "RPLM_acceleration_Y_component",
    "RPLM_acceleration_Z_component",
    "RPLM_jerk_magnitude",
    "RPLM_jerk_X_component",
    "RPLM_jerk_Y_component",
    "RPLM_jerk_Z_component",
    "LPLM_speed_magnitude",
    "LPLM_speed_X_component",
    "LPLM_speed_Y_component",
    "LPLM_speed_Z_component",
    "LPLM_acceleration_magnitude",
    "LPLM_acceleration_X_component",
    "LPLM_acceleration_Y_component",
    "LPLM_acceleration_Z_component",
    "LPLM_jerk_magnitude",
    "LPLM_jerk_X_component",
    "LPLM_jerk_Y_component",
    "LPLM_jerk_Z_component",
]

In [3]:
def f1scoremargin(ground_truth, predictions, tolerance):
    """
    Calcola l'F1 score con una finestra di tolleranza sui change points.
    
    :param ground_truth: Lista o array di change points reali
    :param predictions: Lista o array di change points predetti
    :param tolerance: La tolleranza temporale (numero di unità temporali)
    :return: precision, recall, f1-score
    """
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)
    
    # Vettori per tracciare quali punti sono stati già associati
    matched_ground_truth = np.zeros(len(ground_truth), dtype=bool)
    matched_predictions = np.zeros(len(predictions), dtype=bool)

    mgt={key: False for key in ground_truth}
    mcp={key: False for key in predictions}
    #print(f'gt:{len(ground_truth)} - cp:{len(predictions)}')
    # True Positives (TP)
    tp = 0
    for i, gt_point in enumerate(ground_truth):
        for j, pred_point in enumerate(predictions):
            
            if not matched_predictions[j] and abs(gt_point - pred_point) <= tolerance:
                tp += 1
                matched_ground_truth[i] = True
                matched_predictions[j] = True

                mgt[gt_point] = True
                mcp[pred_point] = True
                break
            
    
    # False Positives (FP) - predizioni non corrispondenti a nessun ground truth entro la tolleranza
    fp = np.sum(~matched_predictions)
    
    # False Negatives (FN) - punti del ground truth non corrispondenti a nessuna predizione entro la tolleranza
    fn = np.sum(~matched_ground_truth)
    #print(f'tp:{tp} - fp:{fp} - fn:{fn}')
    #print(mgt)
    #print(mcp)
    # Calcolo di precision, recall e F1-score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    #print(f'gt:{len(ground_truth)} cp:{len(predictions)} tp:{tp} fp:{fp} fn:{fn}')
    return precision, recall, f1, {"tp":tp, "fp":fp, "fn":fn}

In [4]:
def ReadAndPreProcess(inputDataRaw):
    # lettura
    df=pd.read_csv(inputDataRaw,sep=' ', header=None).interpolate()
    
    df=df.drop(0, axis=1)
    df=df.drop_duplicates()
    df = df.iloc[:, ::-1]
 

    return df

In [5]:
# questa funzione ritorna un dataframe del groundtruth che viene usato specificatamente per visualizzare il gt
# è soggetto a un preprocessing dei dati siccome l'ultimo groundtruth è dove termina il ts del gt
# di conseguenza per farlo corrispondere, bisogna stretcharlo
# ma ricordo di aver rifatti i dati nuovi per generare un groundtruth a fine ts, da controllare cosi che non serve stretcharlo?
def LoadingGroundTruth(df,gtraw):
    gt=pd.read_csv(gtraw,sep=' ', header=None)
    gt=gt.iloc[:,0].values
    #stretching dei dati se necessario per farlo corrispondere alla ts dei dati
    stretch_gt = np.array([])
    for idx,i in enumerate(gt):
        relpos = len(df)*i/gt[-1]
        stretch_gt = np.append(stretch_gt,relpos)

    # eliminiamo l'ultimo elemento che è stato annotato solo per delimitare la lunghezza della gt simile alla ts
    
    return stretch_gt[:-1]

In [6]:
def GetClasp2(df,gt,feature, p_idx, v_idx,**kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for i in feature:
        print(f'computing: {p_idx} / {v_idx} / {i}')
        ts=df.iloc[:,i]
        print(ts.values)
        
        
        clasp = BinaryClaSPSegmentation(**kwargs)
            
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        # WTF IS THIS
        """
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1
        """
        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        #potenziale bug.
        #se faccio unique() mi toglie il numero di cp in un punto e quando faccio majority voting mi si toglie
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        

        
    return result, eachresult, eachclasp

In [7]:
# utilizzo CLASP 
# prende come parametro un dataframe e restituisce il clasp score
# gt e known vengono usati per usare il numero vero di cp se uguale a 1 sennò si cerca di predirlo se il modello lo permette
def GetClasp3(df,gt,known,feature, **kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for i in [feature]:
    
        ts=df.iloc[:,i]
        
        #print(ts.head())
        if known == 1:
            #print("knwon!")
            clasp = BinaryClaSPSegmentation(n_segments=len(gt), validation=None)
        else:
            #print("unknown!")
            clasp = BinaryClaSPSegmentation(**kwargs)
            
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        # WTF IS THIS
        """
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1
        """

        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        

        
    return result, eachresult, eachclasp

In [8]:
# utilizzo CLASP 
# prende come parametro un dataframe e restituisce il clasp score
# gt e known vengono usati per usare il numero vero di cp se uguale a 1 sennò si cerca di predirlo se il modello lo permette
def GetClasp(df,gt,known, **kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for i in range(0,len(features_name)):
    
        ts=df.iloc[:,i]
        
        #print(ts.head())
        if known == 1:
            #print("knwon!")
            clasp = BinaryClaSPSegmentation(n_segments=len(gt), validation=None)
        else:
            #print("unknown!")
            clasp = BinaryClaSPSegmentation(**kwargs)
            
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        # WTF IS THIS
        """
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1
        """
        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        

        
    return result, eachresult, eachclasp
        
    


In [9]:
def FPremoverDTW(featureTS,res,threshold):
     # controlla quale segmento piu lungo o corto
    # si fa una finestra grande quanto corto e slitta su lungo
    # si fa un dtw per ogni finestra e si prende lo score piu basso
    # se entro un certo limite si unisce, senno si continua
    if len(res) < 3:
        return res
    to_color=[]
    filtered=[res[0]]
    lo = 0
    hi = 1
    while hi < len(res)-1:
       # print(f'hi:{hi} max:{len(res)}')
        a = featureTS[int(res[lo]):int(res[hi])]
        b = featureTS[int(res[hi]):int(res[hi+1])]
        # controllo quale segmento piu lungo o corto
        # halflo = segmento piu corto è una timeseries
        # halfhi = segmento piu lungo è una timeseries
        if len(a) < len(b):
            halflo=featureTS[int(res[lo]):int(res[hi])]
            halfhi=featureTS[int(res[hi]):int(res[hi+1])]
        else:
            halfhi=featureTS[int(res[lo]):int(res[hi])]
            halflo=featureTS[int(res[hi]):int(res[hi+1])]

        # faccio sliding window e trovo la distance piu piccola
        st = 0
        en = len(halflo) # possibile bug
        smallestdist=-1
        smalllo=[]
        highlo=[]
      
       # print(f'en:{en} whilelim:{len(halfhi)}')
        while en < len(halfhi):
            
            slidewin = halfhi[st:en]
            halflo_d = np.array(halflo, dtype=np.double)
            slidewin_d = np.array(slidewin, dtype=np.double)
            distance = dtw.distance_fast(halflo_d, slidewin_d, use_pruning=True)
            if smallestdist == -1:
                smallestdist = distance
                smalllo=halflo
                highlo=slidewin
            elif smallestdist > distance:
                smallestdist = distance
                smalllo=halflo
                highlo=slidewin

            st+=30
            en+=30
            #print(f'en:{en} lim:{len(halfhi)} dist:{distance}')
        
        #plt.figure(figsize=(18,9))
        #plt.plot(np.arange(len(smalllo)),smalllo,"blue")
       # plt.figure(figsize=(18,9))
       # plt.plot(np.arange(len(highlo)),highlo,"green")
        #plt.xlabel(f'smallest:{smallestdist}')
        if smallestdist <= threshold:
            to_color.append(hi)
            hi+=1
        else:
            filtered.append(res[hi])
            lo=hi
            hi=lo+1
    filtered.append(res[-1])
    return filtered

In [10]:
timeseries=[
    "in\cora1_in.txt",
      "in\cora4_05_in.txt",
      "in\cora4_08_in.txt",
      "in\cora5_in.txt",
      "in\cora14_in.txt",
      "in\marianne7_in.txt",
      "in\marianne8_in.txt",
      "in\marianne10_in.txt",
      "in\marianne18_in.txt",
      "in\marianne19_in.txt",
      "in\marianne24_in.txt",
      "in\marianne26_in.txt",
      "in\marianne41_in.txt",
      "in\marianne42_in.txt",
      "in\marianne43_in.txt",
      "in\marianne47_in.txt",
      "in\marianne48_in.txt",
      "in\muriel18_in.txt",
      "in\muriel26_in.txt",
      "in\muriel27_in.txt",
      "in\muriel30_in.txt"

      ]
groundtruth=[
         "gt\cora_gt_2019-08-08_t001_video01.txt",
         "gt\cora_gt_2019-05-22_t004_video01.txt",
         "gt\cora_gt_2019-08-08_t004_video01.txt",
         "gt\cora5_gt.txt",
         "gt\cora_gt_2019-08-08_t014_video01.txt",
         "gt\marianne_gt_2016-03-22_t007_video01.txt",
         "gt\marianne_gt_2016-03-22_t008_video01.txt",
         "gt\marianne_gt_2016-03-22_t010_video01.txt",
         "gt\marianne_gt_2016-03-22_t018_video01.txt",
         "gt\marianne_gt_2016-03-22_t019_video01.txt",
         "gt\marianne_gt_2016-03-22_t024_video01.txt",
         "gt\marianne_gt_2016-03-22_t026_video01.txt",
         "gt\marianne_gt_2016-03-22_t041_video01.txt",
         "gt\marianne_gt_2016-03-22_t042_video01.txt",
         "gt\marianne_gt_2016-03-22_t043_video01.txt",
         "gt\marianne_gt_2016-03-22_t047_video01.txt",
         "gt\marianne_gt_2016-03-22_t048_video01.txt",
         "gt\muriel_gt_2016-03-21_t018_video01.txt",
         "gt\muriel_gt_2016-03-21_t026_video01.txt",
         "gt\muriel_gt_2016-03-21_t027_video01.txt",
         "gt\muriel_gt_2016-03-23_t030_video01.txt"
         ]



#suss
with open("../dfl.pkl", "rb") as f:
    dfl = pickle.load(f)

with open("../gtl.pkl", "rb") as f:
    gtl = pickle.load(f)

with open("../cpsl.pkl", "rb") as f:
    cpsl = pickle.load(f)

with open("../cpsl_suss_DTW.pkl", "rb") as f:
    cpsl_dtw = pickle.load(f)

#normalized
with open("../dfl_no.pkl", "rb") as f:
    dfl_no = pickle.load(f)

with open("../cpsl_no.pkl", "rb") as f:
    cpsl_no = pickle.load(f)

with open("../cpsl_no_DTW.pkl", "rb") as f:
    cpsl_no_dtw = pickle.load(f)

#standardized
with open("../dfl_st.pkl", "rb") as f:
    dfl_st = pickle.load(f)

with open("../cpsl_st.pkl", "rb") as f:
    cpsl_st = pickle.load(f)

with open("../cpsl_st_DTW.pkl", "rb") as f:
    cpsl_st_dtw = pickle.load(f)


<h1><b>FOR COMPUTING THE PREDICTION</b></h1>


# compute all cp for all timeseries, so later we can use it freely
dfl_st_z=[]
gtl_st_z=[]
cpsl_st_z=[]
for i in range(len(timeseries)):
    print(f'PRIMO LOOP:{i}/{len(timeseries)}')
    df_raw=ReadAndPreProcess(timeseries[i])
    scaler = StandardScaler()
    df_scaler = scaler.fit_transform(df_raw)
    df = pd.DataFrame(df_scaler)

    gt=LoadingGroundTruth(df,groundtruth[i])
    cp_all,cps,clasp=GetClasp2(df,gt,0,list(range(len(features_name))), window_size="suss",distance="euclidean_distance",n_jobs=6)
    dfl_st_z.append(df)
    gtl_st_z.append(gt)
    cpsl_st_z.append(cps)

with open("../dfl_st.pkl", "wb") as f:
    pickle.dump(dfl_st_z, f)

with open("../gtl_st.pkl", "wb") as f:
    pickle.dump(gtl_st_z, f)

with open("../cpsl_st.pkl", "wb") as f:
    pickle.dump(cpsl_st_z, f)

print("COMPLETATO PRIMA PARTE")

# compute all cp for all timeseries, so later we can use it freely
dfl_no_z=[]
gtl_no_z=[]
cpsl_no_z=[]
for i in range(len(timeseries)):
    print(f'SECONDO LOOP:{i}/{len(timeseries)}')
    df_raw=ReadAndPreProcess(timeseries[i])
    normalizer = MinMaxScaler()
    df_normalizer = normalizer.fit_transform(df_raw)
    df=pd.DataFrame(df_normalizer)
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp_all,cps,clasp=GetClasp2(df,gt,0,list(range(len(features_name))), window_size="suss",distance="euclidean_distance",n_jobs=6)
    dfl_no_z.append(df)
    gtl_no_z.append(gt)
    cpsl_no_z.append(cps)

with open("../dfl_no.pkl", "wb") as f:
    pickle.dump(dfl_no_z, f)

with open("../gtl_no.pkl", "wb") as f:
    pickle.dump(gtl_no_z, f)

with open("../cpsl_no.pkl", "wb") as f:
    pickle.dump(cpsl_no_z, f)

<h1><b>FOR FILTERING WITH DTW</b></h1>

new_cpsl=[]
for i in range(len(timeseries)):
    print(f"computing: {i}/{len(timeseries)}")
    df=dfl_no[i]
    gt=gtl[i]
    cps=np.array(cpsl_no[i],dtype="object")
    new_cps=[]
    print("feature: ",end='')
    for idx,cp in enumerate(cps):
        
        print(f'{idx} ',end='')
        new_cp=FPremoverDTW(df.iloc[:,idx].values,cp)
        new_cps.append(new_cp)
    print("")
    new_cpsl.append(new_cps)

with open("../cpsl_no_DTW.pkl", "wb") as f:
    pickle.dump(new_cpsl, f)

new_cpsl=[]
for i in range(len(timeseries)):
    print(f"computing: {i}/{len(timeseries)}")
    df=dfl_st[i]
    gt=gtl[i]
    cps=np.array(cpsl_st[i],dtype="object")
    new_cps=[]
    print("feature: ",end='')
    for idx,cp in enumerate(cps):
        
        print(f'{idx} ',end='')
        new_cp=FPremoverDTW(df.iloc[:,idx].values,cp)
        new_cps.append(new_cp)
    print("")
    new_cpsl.append(new_cps)

with open("../cpsl_st_DTW.pkl", "wb") as f:
    pickle.dump(new_cpsl, f)

In [11]:
def UnionCPS(*val):
    res=np.array([])
    for i in val:
        res = np.append(res,i).flatten()
    return np.sort(res)

In [12]:
def MajorityVoteCP(arr,margin,amount):
    if len(arr)==0:
        return np.array([])
    # se vicino continua ad aggiungere

    # se lontano e bucket presente, generare medio

    # se lontano e bucket vuoto aggiornare start
    bucket=[]
    answer=[]

    for i in range(len(arr)-1,-1,-1):
        if bucket == []:
            bucket.append(arr[i])
        elif abs(arr[i]-bucket[-1]) <= margin:
            bucket.append(arr[i])
        elif abs(arr[i]-bucket[-1]) > margin:
            if len(bucket) < amount:
                bucket=[arr[i]]
            else:
                summ=0
                for j in bucket:
                    summ+=j
                answer.append(summ/len(bucket))
                bucket=[arr[i]]
    if len(bucket) < amount:
        bucket=[]
    else:
        summ=0
        for j in bucket:
            summ+=j
        answer.append(summ/len(bucket))
        bucket=[]
    return np.array(answer)


In [13]:
def delnear(arr,range):
    i = 0
    while i < len(arr) - 1:
        # Iniziamo con il primo elemento di un potenziale gruppo
        gruppo_inizio = i
        gruppo_fine = i

        # Cerca gli elementi che fanno parte dello stesso gruppo
        while gruppo_fine < len(arr) - 1 and arr[gruppo_fine + 1] - arr[gruppo_fine] < range:
            gruppo_fine += 1

        # Se esiste un gruppo di più elementi
        if gruppo_fine > gruppo_inizio:
            # Se la distanza tra l'inizio e la fine è minore di 50, elimina l'elemento maggiore (gruppo_fine)
            if arr[gruppo_fine] - arr[gruppo_inizio] < range:
                arr = np.delete(arr, gruppo_fine)
            
            # Elimina tutti gli elementi interni al gruppo
            arr = np.concatenate((arr[:gruppo_inizio + 1], arr[gruppo_fine:]))

        # Procedi con il prossimo gruppo
        i = gruppo_inizio + 1

    return arr

In [14]:
def IgnoreZone(idx,cpraw,gt):
    cp = cpraw.tolist()
    if idx == 0: #cora1
        for i in range(len(cp)-1, -1, -1):
            #if cp[i] > 3944.7118557910376+100 and cp[i] < 5911.693516853054-100 or cp[i] > 12845.0+100:
            if cp[i] > gt[18]+100 and cp[i] < gt[19]-100 or cp[i]> gt[-1]+100:
                cp.pop(i)
                
        
    elif idx == 1: #cora4_05
        for i in range(len(cp)-1, -1, -1):
            #if cp[i]< 969.6180827886711-100 and cp[i] > 13125.469063180826+100:
            if cp[i] < gt[0]-100 or cp[i] > gt[-1]+100:
                cp.pop(i)
    elif idx == 2: #cora4_08
        for i in range(len(cp)-1, -1, -1):
            #if cp[i] > 2874.607407407407+100 and cp[i] < 4016.935849056604-100:
            if cp[i] > gt[-2]+100 and cp[i] < gt[-1]-100:
                cp.pop(i)

    elif idx == 17: #muriel18
        for i in range(len(cp)-1, -1, -1):
            #if cp[i] > 180.03455207940698+100 and cp[i] < 1227.051137077522-100 or cp[i] > 5865.505591154668+100:
            if cp[i] > gt[0]+100 and cp[i] < gt[1]-100 or cp[i] > gt[-1]+100:
                cp.pop(i)

    elif idx == 18: #muriel26
        for i in range(len(cp)-1, -1, -1):
            #if cp[i] > 138.33224102186853+100 and cp[i] < 3677.231833076974-100:
            if cp[i] > gt[0]+100 and cp[i] < gt[1]-100:
                cp.pop(i)


    elif idx == 21: #muriel30
        for i in range(len(cp)-1, -1, -1):
            #if cp[i] > 8187.634803581529+100:
            if cp[i] > gt[26]+100 and cp[i] < gt[27]-100 or cp[i] > gt[-1]+100:
                cp.pop(i)


    else:
        pass
        #print("error IgnoreZone()")
    return np.array(cp)

In [15]:
# Dato una serie di cp, li combina in uno e fa il majority voting
def AndSal(margin,majvote,*args):
    result=np.array([])
 
    for cp in args:
        result = np.append(result,cp).flatten()
    result = np.sort(result)
    result = MajorityVoteCP(result,margin,majvote)
    return result


In [16]:
def UnionCP(cps,*val):
    res=np.array([])
    for i in val:
        res = np.append(res,cps[i]).flatten()
    return np.sort(res)

In [17]:
def together(cps,div):
    ariel_el=[26,27,28,29,30,31,32,33,34,35,36,37,2,8,14]
    ariel = UnionCPS(*cps[ariel_el])

    strn_el=[38,39,40,41,42,43,44,45,46,47,48,49,1,9]
    strn = UnionCPS(*cps[strn_el])

    rhel_el=[50,51,52,53,54,55,56,57,58,59,60,61,7,11,18,21]
    rhel = UnionCPS(*cps[rhel_el])

    lhel_el=[6,10,19,20,62,63,64,65,66,67,68,69,70,71,72,73]
    lhel = UnionCPS(*cps[lhel_el])
    
    rplm_el=[74,75,76,77,78,79,80,81,82,83,84,85,5,13,23,25]
    rplm = UnionCPS(*cps[rplm_el])

    lplm_el=[4,12,22,24,86,87,88,89,90,91,92,93,94,95,96,97]
    lplm = UnionCPS(*cps[lplm_el])

    
    ariel = MajorityVoteCP(ariel,100,div)#math.ceil(len(ariel_el)/div))
    strn = MajorityVoteCP(strn,100,div)#math.ceil(len(strn_el)/div))
    rhel = MajorityVoteCP(rhel,100,div)#math.ceil(len(rhel_el)/div))
    lhel = MajorityVoteCP(lhel,100,div)#math.ceil(len(lhel_el)/div))
    rplm = MajorityVoteCP(rplm,100,div)#math.ceil(len(rplm_el)/div))
    lplm = MajorityVoteCP(lplm,100,div)#math.ceil(len(lplm_el)/div))

    res = UnionCPS(ariel,strn,rhel,lhel,rplm,lplm)
    res = IgnoreZone(i,res,gt)
    res = delnear(res,100)
    
    _,_,f1,_=f1scoremargin(gt,res,100)
    return f1


In [18]:
def Comp4(cps,k,h):
    vel=cps[[26,27,28,29]]
    acc=cps[[30,31,32,33]]
    jerk=cps[[34,35,36,37]]
    ariel = AndSal(100,h,*AndSal(100,k,*vel),*AndSal(100,k,*acc),*AndSal(100,k,*jerk),UnionCP(cps,2,8,14))

    vel=cps[[38,39,40,41]]
    acc=cps[[42,43,44,45]]
    jerk=cps[[46,47,48,49]]
    strn = AndSal(100,h,*AndSal(100,k,*vel),*AndSal(100,k,*acc),*AndSal(100,k,*jerk),UnionCP(cps,1,9))
    
    vel=cps[[50,51,52,53]]
    acc=cps[[54,55,56,57]]
    jerk=cps[[58,59,60,61]]
    rhel=AndSal(100,h,*AndSal(100,k,*vel),*AndSal(100,k,*acc),*AndSal(100,k,*jerk),UnionCP(cps,7,11,18,21))

    vel=cps[[62,63,64,65]]
    acc=cps[[66,67,68,69]]
    jerk=cps[[70,71,72,73]]
    lhel=AndSal(100,h,*AndSal(100,k,*vel),*AndSal(100,k,*acc),*AndSal(100,k,*jerk),UnionCP(cps,6,10,19,20))

    vel=cps[[74,75,76,77]]
    acc=cps[[78,79,80,81]]
    jerk=cps[[82,83,84,85]]
    rplm=AndSal(100,h,*AndSal(100,k,*vel),*AndSal(100,k,*acc),*AndSal(100,k,*jerk),UnionCP(cps,5,13,23,25))

    vel=cps[[86,87,88,89]]
    acc=cps[[90,91,92,93]]
    jerk=cps[[94,95,96,97]]
    lplm=AndSal(100,h,*AndSal(100,2,*vel),*AndSal(100,2,*acc),*AndSal(100,2,*jerk),UnionCP(cps,4,12,22,24))

    # unisco il tutto in un unico array
    final = UnionCPS(ariel,strn,rhel,lhel,rplm,lplm)
    final = delnear(final,100)
    final = IgnoreZone(i,final,gt)

    _,_,f1,_=f1scoremargin(gt,final,100)

    return f1


In [19]:
def test(lol,**kwargs):
    print(lol)
    print(kwargs)
    print(**kwargs)


In [20]:
param_grid = {
    "window_size": ['suss'],
    "distance": ['euclidean_distance'],
    "score":['roc_auc','f1'],
    "early_stopping": [True,False],
    "k_neighbours":[1,3,5,7,9],
    "n_jobs":[4]
}


# 21*98 = 2058 timeseries
for p_idx,params in enumerate(ParameterGrid(param_grid)):
    if p_idx < 13:
        print(f'skipped:{p_idx}')
        continue
    params["excl_radius"]=params["k_neighbours"]+2
    cpsl=[]
    for i in range(len(timeseries)):
        df=ReadAndPreProcess(timeseries[i])
        gt=LoadingGroundTruth(df,groundtruth[i])
        #cp_all,cps,clasp=GetClasp2(df,gt,0,list(range(len(features_name))), window_size="suss",distance="euclidean_distance",n_jobs=6)
        cp_all,cps,clasp=GetClasp2(df,gt,list(range(len(features_name))),p_idx,i, **params)
        cpsl.append(cps)


    with open("../cpsl_param_"+str(p_idx)+".pkl", "wb") as f:
        pickle.dump(cpsl, f)


skipped:0
skipped:1
skipped:2
skipped:3
skipped:4
skipped:5
skipped:6
skipped:7
skipped:8
skipped:9
skipped:10
skipped:11
skipped:12
computing: 13 / 0 / 0
[0.       0.011978 0.011978 ... 0.07526  0.072244 0.067064]


C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\claspy\nearest_neighbour.py:252: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  start, end = pranges[idx]


computing: 13 / 0 / 1
[0.       0.       0.       ... 0.130845 0.123262 0.120738]
computing: 13 / 0 / 2
[0.       0.       0.       ... 0.755605 0.757314 0.759079]
computing: 13 / 0 / 3
[430.243661 430.171766 430.171766 ... 540.41448  540.433874 540.452042]
computing: 13 / 0 / 4
[0.       0.       0.       ... 0.874497 0.817759 0.765728]
computing: 13 / 0 / 5
[0.       0.       0.       ... 0.335698 0.310352 0.28195 ]
computing: 13 / 0 / 6
[0.       0.       0.       ... 0.00088  0.00072  0.000796]
computing: 13 / 0 / 7
[0.       0.       0.       ... 0.003527 0.003608 0.003692]
computing: 13 / 0 / 8
[0.       0.       0.       ... 0.314741 0.311076 0.308231]
computing: 13 / 0 / 9
[393.78201  393.78872  393.78872  ... 399.8061   399.795541 399.80684 ]
computing: 13 / 0 / 10
[370.462323 370.464987 370.464987 ... 376.353524 376.359062 376.38521 ]
computing: 13 / 0 / 11
[374.056052 374.062478 374.062478 ... 382.164126 382.157933 382.189832]
computing: 13 / 0 / 12
[184.66781  184.570526 18

C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:9205: RuntimeWarning: invalid value encountered in scalar divide
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


computing: 13 / 1 / 4
[0.       0.       0.114331 ... 0.034603 0.052153 0.07333 ]
computing: 13 / 1 / 5
[0.       0.       0.051345 ... 0.075474 0.102897 0.136134]
computing: 13 / 1 / 6
[0.000e+00 0.000e+00 1.100e-05 ... 2.050e-03 3.274e-03 4.134e-03]
computing: 13 / 1 / 7
[0.000e+00 0.000e+00 6.000e-06 ... 1.239e-03 1.358e-03 1.487e-03]
computing: 13 / 1 / 8
[0.       0.       0.000137 ... 0.049399 0.046634 0.043339]
computing: 13 / 1 / 9
[429.484735 429.464587 429.457439 ... 418.545842 418.563528 418.566474]
computing: 13 / 1 / 10
[376.328855 376.34296  376.299998 ... 372.364844 372.388826 372.388132]
computing: 13 / 1 / 11
[373.166347 373.172705 373.177387 ... 371.608372 371.642433 371.648778]
computing: 13 / 1 / 12
[187.531256 187.683545 188.230672 ... 271.975799 271.972842 271.987583]
computing: 13 / 1 / 13
[191.391623 191.66875  192.133389 ... 263.189159 263.202054 263.258976]
computing: 13 / 1 / 14
[153.624315 153.590118 153.488329 ... 151.903326 151.903249 151.896671]
computing

...

computing: 17 / 18 / 10
[388.572765 388.566703 388.608459 ... 385.583688 385.566843 385.562854]
computing: 17 / 18 / 11
[386.214669 386.202463 386.260627 ... 387.567371 387.602568 387.638763]
computing: 17 / 18 / 12
[291.854949 291.87402  291.92285  ... 296.592155 296.485124 296.3691  ]
computing: 17 / 18 / 13
[288.570845 288.571315 288.687927 ... 286.570603 286.560942 286.538014]
computing: 17 / 18 / 14
[147.649873 147.640602 147.725563 ... 150.409346 150.420289 150.441969]
computing: 17 / 18 / 15
[ 7.813312  7.813312 10.210543 ... 11.334835 11.341779 11.341779]
computing: 17 / 18 / 16
[17.210725 17.210725 20.269639 ... 18.372276 18.406812 18.406812]
computing: 17 / 18 / 17
[0.537529 0.537529 2.553801 ... 6.210732 6.241347 6.241347]
computing: 17 / 18 / 18
[170.838235 170.838235 169.460547 ... 168.583344 168.593067 168.593067]
computing: 17 / 18 / 19
[171.265262 171.265262 171.739618 ... 173.111803 173.205495 173.205495]
computing: 17 / 18 / 20
[10.156673 10.156673  9.782673 ...  8.85

C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:9205: RuntimeWarning: invalid value encountered in scalar divide
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


computing: 18 / 1 / 4
[0.       0.       0.114331 ... 0.034603 0.052153 0.07333 ]
computing: 18 / 1 / 5
[0.       0.       0.051345 ... 0.075474 0.102897 0.136134]
computing: 18 / 1 / 6
[0.000e+00 0.000e+00 1.100e-05 ... 2.050e-03 3.274e-03 4.134e-03]
computing: 18 / 1 / 7
[0.000e+00 0.000e+00 6.000e-06 ... 1.239e-03 1.358e-03 1.487e-03]
computing: 18 / 1 / 8
[0.       0.       0.000137 ... 0.049399 0.046634 0.043339]
computing: 18 / 1 / 9
[429.484735 429.464587 429.457439 ... 418.545842 418.563528 418.566474]
computing: 18 / 1 / 10
[376.328855 376.34296  376.299998 ... 372.364844 372.388826 372.388132]
computing: 18 / 1 / 11
[373.166347 373.172705 373.177387 ... 371.608372 371.642433 371.648778]
computing: 18 / 1 / 12
[187.531256 187.683545 188.230672 ... 271.975799 271.972842 271.987583]
computing: 18 / 1 / 13
[191.391623 191.66875  192.133389 ... 263.189159 263.202054 263.258976]
computing: 18 / 1 / 14
[153.624315 153.590118 153.488329 ... 151.903326 151.903249 151.896671]
computing

C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:9205: RuntimeWarning: invalid value encountered in scalar divide
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


computing: 19 / 1 / 4
[0.       0.       0.114331 ... 0.034603 0.052153 0.07333 ]
computing: 19 / 1 / 5
[0.       0.       0.051345 ... 0.075474 0.102897 0.136134]
computing: 19 / 1 / 6
[0.000e+00 0.000e+00 1.100e-05 ... 2.050e-03 3.274e-03 4.134e-03]
computing: 19 / 1 / 7
[0.000e+00 0.000e+00 6.000e-06 ... 1.239e-03 1.358e-03 1.487e-03]
computing: 19 / 1 / 8
[0.       0.       0.000137 ... 0.049399 0.046634 0.043339]
computing: 19 / 1 / 9
[429.484735 429.464587 429.457439 ... 418.545842 418.563528 418.566474]
computing: 19 / 1 / 10
[376.328855 376.34296  376.299998 ... 372.364844 372.388826 372.388132]
computing: 19 / 1 / 11
[373.166347 373.172705 373.177387 ... 371.608372 371.642433 371.648778]
computing: 19 / 1 / 12
[187.531256 187.683545 188.230672 ... 271.975799 271.972842 271.987583]
computing: 19 / 1 / 13
[191.391623 191.66875  192.133389 ... 263.189159 263.202054 263.258976]
computing: 19 / 1 / 14
[153.624315 153.590118 153.488329 ... 151.903326 151.903249 151.896671]
computing

cleaned_array = [s[3:-7] for s in timeseries]
resexcel=pd.DataFrame()
resexcel["name"] = cleaned_array
# calcolo per ogni valore di threshold di fpremover e saranno le mie colonne
thre=[2,4,6,8,10,14,18,25]
cps_thre=[]
for t in thre:
    print(f'threshold:{t}')
    new_cpsl=[]
    for i in range(len(timeseries)):
        df=dfl_st[i]
        gt=gtl[i]
        cps=np.array(cpsl_st[i],dtype="object")
        # FILTRO CPS
        new_cps=[]
        print("feature: ",end='')
        # per ogni feature, applico fpremover per ottenere nuovi cp con meno FP
        for idx,cp in enumerate(cps):
            
            print(f'{idx} ',end='')
            new_cp=FPremoverDTW(df.iloc[:,idx].values,cp,t)
            new_cps.append(new_cp)

        new_cpsl.append(new_cps)

    cps_thre.append(new_cpsl)

for idx,cpslt in enumerate(cps_thre):
    # per ogni riga, per ogni video, calcolo nuovi cp e calcolo lo score
    for div in range(1,4):
        f1_list_together=[]
        f1_list_comp4=[]
        print(f'div:{div}')
        for i in range(len(timeseries)):
            print(f"computing: {i}/{len(timeseries)}",end="\r")
            # prendo i dati standardizzati
            df=dfl_st[i]
            gt=gtl[i]
            cps=np.array(cpslt[i],dtype="object")
           
            #calcolare together
            f1_list_together.append(together(np.array(cps,dtype="object"),div))

            #calcolare comp4
            f1_list_comp4.append(Comp4(np.array(cps,dtype="object"),div,1))

        print(f"computing: {len(timeseries)}/{len(timeseries)}")
        resexcel[f"t:{idx} d:{div}"]=f1_list_together
resexcel.to_excel("outputFile/together&comp4_FPremoverDTW_before_standardized_thresholdStudy.xlsx")
    
        


cleaned_array = [s[3:-7] for s in timeseries]
resexcel=pd.DataFrame()
resexcel["name"] = cleaned_array
# calcolo per ogni valore di threshold di fpremover e saranno le mie colonne
thre=[0.2,0.4,0.8,1.0,1.4,1.8,2.5,3.0]
cps_thre=[]
for t in thre:
    print(f'threshold:{t}')
    new_cpsl=[]
    for i in range(len(timeseries)):
        df=dfl_no[i]
        gt=gtl[i]
        cps=np.array(cpsl_no[i],dtype="object")
        # FILTRO CPS
        new_cps=[]
        print("feature: ",end='')
        # per ogni feature, applico fpremover per ottenere nuovi cp con meno FP
        for idx,cp in enumerate(cps):
            
            print(f'{idx} ',end='')
            new_cp=FPremoverDTW(df.iloc[:,idx].values,cp,t)
            new_cps.append(new_cp)
        new_cpsl.append(new_cps)
    cps_thre.append(new_cpsl)


for idx,cpslt in enumerate(cps_thre):
    # per ogni riga, per ogni video, calcolo nuovi cp e calcolo lo score
    for div in range(1,4):
        f1_list_together=[]
        f1_list_comp4=[]
        print(f'div:{div}')
        for i in range(len(timeseries)):
            print(f"computing: {i}/{len(timeseries)}",end="\r")
            # prendo i dati standardizzati
            df=dfl_st[i]
            gt=gtl[i]
            cps=np.array(cpslt[i],dtype="object")

            #calcolare together
            f1_list_together.append(together(np.array(cps,dtype="object"),div))

            #calcolare comp4
            f1_list_comp4.append(Comp4(np.array(cps,dtype="object"),div,1))

        print(f"computing: {len(timeseries)}/{len(timeseries)}")
        resexcel[f"t:{idx} d:{div}"]=f1_list_together
resexcel.to_excel("outputFile/together&comp4_FPremoverDTW_before_normalized_thresholdStudy.xlsx")